#Майнор ИАД. Домашнее задание 3. YOLO.

В этом задании вы напишете и обучите свой собственный YOLO детектор. Нужно будет разобраться со статьей: понять какого формата должна быть обучающая пара (x, y), как перевести лосс из математической формулы в питоновский код - ну и конечно понять и реализовать саму архитектуру модели.

Выборка на котрой мы будем обучать модель состоит из разнообразных фотографий яблок, бананов и апельсинов. Данные скачиваем [отсюда](https://drive.google.com/file/d/1d8GSfZoWbraWCSUhX78yl4CnMFYE-5n3/view?usp=sharing).

Баллы за ДЗ распределены следующим образом: 
- Выборка для YoloV1 - 2 балла
- YOLO модель - 2 балла
- YOLO Loss - 3 балла
- Вспомогательные функции - 2 балла
- Обучение и расчет метрик - 2 балла

Для построения и обучения можно использовать как pytorch, так и pytorch-lightning.

Да-да, баллов в сумме получается 11

In [ ]:
# Данная библиотека понадобится нам, чтобы обработать разметку
! pip install xmltodict pytorch-lightning

Скачаем данные

In [ ]:
!wget --quiet --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://drive.google.com/uc?export=download&id=1d8GSfZoWbraWCSUhX78yl4CnMFYE-5n3' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1d8GSfZoWbraWCSUhX78yl4CnMFYE-5n3" -O data.zip && rm -rf /tmp/cookies.txt
!unzip -q data.zip
!rm data.zip
!ls -l

Посмотрим как выглядит один из файлов разметки

In [ ]:
!cat data/train/apple_3.xml

## Релизуйте выборку для YoloV1 - 2 балла

In [ ]:
import os
import cv2
import json
import glob
import tqdm
import xmltodict

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import auc

from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
import torch
import torchvision
import pytorch_lightning as pl

import albumentations as A
import albumentations.pytorch

from IPython.core.display import struct
from typing import List
from PIL import Image
import xml.etree.ElementTree as ET

Так как в этом домашнем задании использовать аугментации для обучения __обязательно__ - советуем воспользоваться библиотекой albumentations.

Она  особенно удобна, поскольку умеет сама вычислять новые координаты bounding box'ов после трансформаций картинки. Для знакомства с этим механизмом советуем следующий [гайд](https://albumentations.ai/docs/getting_started/bounding_boxes_augmentation/). 

Вы все еще можете избрать путь torchvision.transforms, вам потребуется знакомый нам метод `__getitem__`, однако вычислять новые координаты bounding box'ов после трансформаций вам придётся вручную

__Обратите внимание__ на то, что в статье коробки предсказаний параметризуются через: _(x_center, y_center, width, height)_ (причем эти значения _относительные_), а в наших файлах - это _(x_min, y_min, x_max, y_max)_

Также, помните что модель должна предсказывать как прямоугольник с обьектом, так и вероятности каждого класса!

In [ ]:
def comparison(data_dir):
    path = os.path.join(os.getcwd(), data_dir)
    answer = {}
    count = 0
    for filename in os.listdir(path):
        name = filename.split('.')[0]
        if name not in answer.keys():
            answer[name] = count
            count += 1
    return answer

def paths(labels, data_dir):
    path = os.path.join(os.getcwd(), data_dir)
    path_im = {}
    path_box = {}
    for name in list(labels.keys()):
        num = labels[name]
        path_im[num] = os.path.join(path, (name + '.jpg'))
        path_box[num] = os.path.join(path, (name + '.xml'))
    return path_im, path_box

In [ ]:
class2tag = {"apple": 1, "orange": 2, "banana": 3}

class FruitDataset(Dataset):
    def __init__(self, data_dir, transforms=None):
        self.biection = comparison(data_dir)
        self.image_paths, self.box_paths = paths(self.biection, data_dir)
        assert len(self.image_paths) == len(self.box_paths)
        self.transforms = transforms

    def __getitem__(self, idx):
        image = np.array(Image.open(self.image_paths[idx]).convert("RGB"))
        bboxes, labels = self.__get_boxes_from_xml(self.box_paths[idx])
        bboxes = (self.__convert_to_yolo_box_params(bboxes, image.shape[1], image.shape[0]))
        if self.transforms:
            transformed = self.transforms(image = image, bboxes = bboxes, labels = labels)
            bboxes = transformed['bboxes']
            image = transformed['image']
            labels = transformed['labels']

        return torch.Tensor(image).permute(2, 0, 1), (bboxes, labels)

        return image, target_tensor

    def __len__(self):
        return len(self.image_paths)

    def __get_boxes_from_xml(self, xml_filename: str):
        boxes = []
        class_labels = []

        for object in ET.parse(xml_filename).getroot().iter('object'):
            name = object.find('name').text
            xmin = int(object.find('bndbox').find('xmin').text)
            ymin = int(object.find('bndbox').find('ymin').text)
            xmax = int(object.find('bndbox').find('xmax').text)
            ymax = int(object.find('bndbox').find('ymax').text)

            boxes.append([xmin, ymin, xmax, ymax])
            class_labels.append(class2tag[name])

        return boxes, class_labels

    def __convert_to_yolo_box_params(self, box_coordinates: List[int], im_w, im_h):
        answers = []
        for box_coordinate in box_coordinates:
            ans = []
            ans.append((box_coordinate[0] + box_coordinate[2]) / 2 / im_w)
            ans.append((box_coordinate[1] + box_coordinate[3]) / 2 / im_h)

            ans.append((box_coordinate[2] - box_coordinate[0]) / im_w)
            ans.append((box_coordinate[3] - box_coordinate[1]) / im_h)
            answers.append(ans)

        return answers

In [ ]:
WIDTH, HEIGHT = 448, 448

train_transform = A.Compose([albumentations.augmentations.geometric.resize.Resize(height=HEIGHT, width = WIDTH),
    A.HorizontalFlip(p=0.3), A.Transpose(p=0.3), A.PixelDropout(p=0.5), A.RandomRotate90(p=0.3),
    A.Blur(blur_limit=(3,4), p=0.5)], bbox_params=A.BboxParams(format='yolo', label_fields=['labels']))

test_transform = A.Compose([albumentations.augmentations.geometric.resize.Resize(height=HEIGHT, width = WIDTH)],
                           bbox_params=A.BboxParams(format='yolo', label_fields=['labels']))

In [ ]:
train_dataset = FruitDataset(
    transforms=train_transform,
    data_dir="./data/train"
    )

val_dataset = FruitDataset(
    transforms=test_transform, 
    data_dir="./data/test"
    )

# Немного проверок, чтобы убедиться в правильности направления решения
assert isinstance(train_dataset[0], tuple)
assert len(train_dataset[0]) == 2
assert isinstance(train_dataset[0][0], torch.Tensor)
print("Тесты успешно пройдены")

Тесты успешно пройдены


In [ ]:
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size= 4,
    shuffle=True)

val_dataloader = DataLoader(
    dataset=val_dataset,
    batch_size=4, 
    shuffle=False
    )

Теперь определим функцию для рассчета Intersection Over Union по 4 углам двух прямоугольников

In [ ]:
def intersection_over_union(predicted_bbox, gt_bbox) -> float:
    """
    Intersection Over Union для двух прямоугольников

    :param: predicted_bbox - [x_min, y_min, x_max, y_max]
    :param: gt_bbox - [x_min, y_min, x_max, y_max]
    
    :return: Intersection Over Union
    """

    intersection_bbox = np.array(
        [
            max(predicted_bbox[0], gt_bbox[0]),
            max(predicted_bbox[1], gt_bbox[1]),
            min(predicted_bbox[2], gt_bbox[2]),
            min(predicted_bbox[3], gt_bbox[3]),
        ]
    )

    intersection_area = max(intersection_bbox[2] - intersection_bbox[0], 0) * max(
        intersection_bbox[3] - intersection_bbox[1], 0
    )
    area_dt = (predicted_bbox[2] - predicted_bbox[0]) * (predicted_bbox[3] - predicted_bbox[1])
    area_gt = (gt_bbox[2] - gt_bbox[0]) * (gt_bbox[3] - gt_bbox[1])

    union_area = area_dt + area_gt - intersection_area

    iou = intersection_area / union_area
    return iou

Теперь начинается основная часть домашнего задания: обучите модель YOLO для object detection на __обучающем__ датасете. 

 - Создайте модель и функцию ошибки YoloV1 прочитав [оригинальную статью](https://paperswithcode.com/paper/you-only-look-once-unified-real-time-object)
 - Напишите функцию обучения модели
 - Используйте аугментации

## Реализуйте Модель - 2 балла

Копировать точное количество слоев и параметры сверток необязательно. Главное - чтобы модель работала по принципу, описанному в статье и делала предсказание в представленном формате.


В качестве подсказки напомним, что выходом модели __для каждого обьекта__ должен быть тензор размера
__S * S * (B * 5 + С)__, где все параметры имеют такое же значение, как и в статье: 

- S - количество ячеек на которое разбивается изображение по вертикали/горизонтали
- В - количество предсказываемых прямоугольников в каждой ячейке
- 5 - количество параметров для определения каждого прямоугольника (x_center, y_center, width, height, confidence)
- С - количество классов (apple, banana, orange)

Таким образом, мы для каждого окна размера __S x S__ предсказываем __В__ коробо и один класс

In [ ]:
class CNNBlock(nn.Module):
    def __init__(self, convs, out_channels, is_max_pool:bool=False, **kwargs):
        
        super().__init__()
        self.conv_layers = convs
        self.batchnorm = nn.BatchNorm2d(out_channels)
        self.leakyrelu = nn.LeakyReLU(0.1)

        self.is_maxpool = is_max_pool
        self.maxpool = nn.MaxPool2d(2,stride=2)
        
    def forward(self, x):
        x = self.conv_layers(x)

        if len(x.size()) == 3:
            x = x.unsqueeze(0)
        x = self.leakyrelu(self.batchnorm(x))

        if self.is_maxpool:
            x = self.maxpool(x)
        return x
    
class YOLO(nn.Module):
    def __init__(self, S=7, B=2, C=3):
        
        super(YOLO, self).__init__()

        self.S = S
        self.B = B
        self.C = C
        self.conv_block_1 = CNNBlock(nn.Sequential(nn.Conv2d(
            in_channels=3, out_channels=192, kernel_size=3, stride=2, padding=1)), 192, True)
        
        self.conv_block_2 = CNNBlock(nn.Sequential(nn.Conv2d(
            in_channels=192, out_channels=256, kernel_size=3, padding=1)), 256, True)
        
        self.conv_block_3 = CNNBlock(nn.Sequential(nn.Conv2d(in_channels=256, out_channels=256, kernel_size=1),
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1)), 512, True)
        
        self.conv_block_4 = CNNBlock(nn.Sequential(nn.Conv2d(
            in_channels=512, out_channels=256, kernel_size=1),
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1),
            nn.Conv2d(in_channels=512, out_channels=256, kernel_size=1),
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=1),
            nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3, padding=1)), 1024, True)
        
        self.conv_block_5 = CNNBlock(nn.Sequential(nn.Conv2d(
            in_channels=1024, out_channels=512, kernel_size=1),
            nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3, padding=1),
            nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=3, stride=2, padding=1)), 1024, False)
        
        self.conv_block_6 = CNNBlock(nn.Sequential(nn.Conv2d(
            in_channels=1024, out_channels=1024, kernel_size=3, padding=1)), 1024, False)
        
        self.linear_1 =  nn.Linear(7*7*1024, 4096)
        self.leaky_relu = nn.LeakyReLU(0.1)
        self.linear_2 =  nn.Linear(4096, S*S*(B*5 + C))

    def forward(self, x):
        x = self.conv_block_1.forward(x)
        x = self.conv_block_2.forward(x)
        x = self.conv_block_3.forward(x)
        x = self.conv_block_4.forward(x)
        x = self.conv_block_5.forward(x)
        x = self.conv_block_6.forward(x)
        x = self.leaky_relu(self.linear_1(nn.Flatten()(x)))
        x = self.linear_2(x).view(self.S,self.S,self.B*5 + self.C)

        return x


testing_model = YOLO()
expected_output_shape = testing_model.S * testing_model.S * (5 * testing_model.B + testing_model.C)

testing_image = train_dataset[0][0]

assert testing_model(testing_image).reshape(-1).shape[0] == expected_output_shape

## Реализуйте YoloLoss - 3 балла

In [ ]:
class YoloLoss(nn.Module):
    def __init__(self, S=7, B=2, C=3):
        """
        :param: S * S - количество ячеек на которые разбивается изображение
        :param: B - количество предсказанных прямоугольников в каждой ячейке
        :param: C - количество классов
        """
        
        super().__init__()
        self.mse = nn.MSELoss(reduction="sum")

        self.S = S
        self.B = B
        self.C = C

        self.lambda_noobj = 0.5
        self.lambda_coord = 5

    def forward(self, predictions, target):
        predictions = predictions.reshape(-1, self.S, self.S, self.C + self.B * 5)

        iou_b1 = intersection_over_union(predictions[..., 21:25], target[..., 21:25])
        iou_b2 = intersection_over_union(predictions[..., 26:30], target[..., 21:25])
        ious = torch.cat([iou_b1.unsqueeze(0), iou_b2.unsqueeze(0)], dim=0)
        iou_max, best_box = torch.max(ious, dim=0)
        exists_box = target[..., 20].unsqueeze(3)

        box_predictions = exists_box * (best_box * predictions[..., 26:30] + (1 - best_box) * predictions[..., 21:25])
        box_targets = exists_box * target[..., 21:25]
        box_predictions[..., 2:4] = torch.sign(box_predictions[..., 2:4]) * torch.sqrt(torch.abs(box_predictions[..., 2:4] + 1e-6))

        box_targets[..., 2:4] = torch.sqrt(box_targets[..., 2:4])

## Реализуйте дополнительные функции из статьи - 2 балла

In [ ]:
def non_max_suppression(bboxes, iou_threshold, threshold):
 

def mean_average_precision(pred_boxes, true_boxes, iou_threshold=0.5):
    ## YOUR CODE
    pass

def get_bound_boxes(loader, model, iou_threshold=.5, threshold=.4):
    ## YOUR CODE
    return all_pred_boxes, all_true_boxes

## Обучите модель и посчитайте метрики для задачи детекции - 2 балла 

Несмотря на то, что в этом блоке ничего сильно нового для вас не ожидается и за него формально дается лишь два балла - провести обучение очень важно для понимания того, насколько правильно реализована ваша модель и лосс.

В процессе обучения будет видно все ли размерности совпадают, падает ли лосс и растут ли метрики целевой задачи, поэтому на практике этот пункт гораздо оказывается гораздо важнее.

In [ ]:
class YOLOLearner(pl.LightningModule):
    def __init__(self) -> None:
        super().__init__()

        self.model = YOLO()
        self.loss = YoloLoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-3)

    def forward(self, x) -> torch.Tensor:
        return self.model(x)

    def configure_optimizers(self):
        return self.optimizer

    def training_step(self, train_batch, batch_idx) -> torch.Tensor:
        ## YOUR CODE
        pass

    def validation_step(self, val_batch, batch_idx) -> None:
        ## YOUR CODE
        pass

In [ ]:
model = # YOUR CODE
n_epochs = # YOUR CODE

yolo_learner = YOLOLearner(...)  ## YOUR CODE

device = "gpu" if torch.cuda.is_available() else "cpu"
trainer = pl.Trainer(accelerator=device, max_epochs=n_epochs)

trainer.fit(yolo_learner, train_dataloader, val_dataloader)

## Посчитайте метрики задачи детекции на валидационной выборке

Попробуйте понять насколько хороши ваши показатели. Если числа кажутся подозрительно низкими - возможно вам стоит перепроверить ваше решение. 

In [ ]:
## YOUR CODE

## Визуализируйте предсказанные bounding box'ы для любых пяти картинок из __валидационного__ датасета.

In [ ]:
image, targets = next(iter(val_dataset))
preds = ## YOUR CODE

In [ ]:
from PIL import ImageDraw

image = torchvision.transform.ToPILImage()(image)
draw = ImageDraw.Draw(image)

for box in targets[0]:
    ## YOUR CODE
    draw.rectangle([(box[0], box[1]), (box[2], box[3])])

for box in preds[0]:
    ## YOUR CODE
    draw.rectangle([(box[0], box[1]), (box[2], box[3])], outline='red')
image